## Import Libraries

In [1]:
import os
import webbrowser
from neo4j import GraphDatabase
from pyvis.network import Network
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image
import time

## Load Data

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../../Encryption/Blank-Sandbox.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")

## Graph Initialisation

In [5]:
driver = GraphDatabase.driver(uri, auth=(username, password))

In [6]:
# Function to create graph data
def create_graph(tx):
    # Create places
    tx.run("MERGE (:Place {city: 'Berlin', country: 'DE'})")
    tx.run("MERGE (:Place {city: 'London', country: 'UK'})")

    # Create people
    tx.run("MERGE (:Person {name: 'Rosa', gender: 'f'})")
    tx.run("MERGE (:Person {name: 'Karl', age: 64})")
    tx.run("MERGE (:Person {name: 'Fred'})")

    # Create relationships
    tx.run("""
    MATCH (p:Person {name: 'Rosa'}), (c:Place {city: 'Berlin'})
    MERGE (p)-[:LIVES_IN {since: 2020}]->(c)
    """)

    tx.run("""
    MATCH (p:Person {name: 'Karl'}), (c:Place {city: 'London'})
    MERGE (p)-[:LIVES_IN {since: 1980}]->(c)
    """)

    tx.run("""
    MATCH (p:Person {name: 'Fred'}), (c:Place {city: 'London'})
    MERGE (p)-[:LIVES_IN]->(c)
    """)

    tx.run("""
    MATCH (a:Person {name: 'Rosa'}), (b:Person {name: 'Karl'})
    MERGE (a)-[:FRIEND]->(b)
    """)

    tx.run("""
    MATCH (a:Person {name: 'Karl'}), (b:Person {name: 'Fred'})
    MERGE (a)-[:FRIEND]->(b)
    """)

    tx.run("""
    MATCH (a:Person {name: 'Fred'}), (b:Person {name: 'Karl'})
    MERGE (a)-[:FRIEND]->(b)
    """)

## Constraints

### Define Constraints

In [7]:
# Create the graph
with driver.session() as session:
    session.execute_write(create_graph)

In [8]:
def create_uniqueness_constraints(tx):
    # Uniqueness constraint for Person.name
    tx.run("CREATE CONSTRAINT unique_person_name IF NOT EXISTS FOR (p:Person) REQUIRE p.name IS UNIQUE")
    
    # Uniqueness constraint for Place.city
    tx.run("CREATE CONSTRAINT unique_place_city IF NOT EXISTS FOR (c:Place) REQUIRE c.city IS UNIQUE")

In [9]:
with driver.session() as session:
    session.execute_write(create_uniqueness_constraints)

### Constraints Visualisation

In [10]:
def show_constraints(tx):
    result = tx.run("SHOW CONSTRAINTS")

    for record in result:
        print(record)

In [11]:
with driver.session() as session:
    session.execute_read(show_constraints)

<Record id=4 name='unique_person_name' type='UNIQUENESS' entityType='NODE' labelsOrTypes=['Person'] properties=['name'] ownedIndex='unique_person_name' propertyType=None>
<Record id=5 name='unique_place_city' type='UNIQUENESS' entityType='NODE' labelsOrTypes=['Place'] properties=['city'] ownedIndex='unique_place_city' propertyType=None>


In [12]:
import pandas as pd
from tabulate import tabulate

In [13]:
def get_constraints(tx):
    result = tx.run("SHOW CONSTRAINTS")
    constraints = []
    for record in result:
        constraints.append({
            "name": record["name"],
            "type": record["type"],
            "entityType": record["entityType"],
            "labelsOrTypes": record["labelsOrTypes"],
            "properties": record["properties"]
        })
    return constraints

In [14]:
with driver.session() as session:
    constraints_data = session.execute_read(get_constraints)

In [15]:
df = pd.DataFrame(constraints_data)

In [16]:
# Pretty print using tabulate
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+--------------------+------------+--------------+-----------------+--------------+
|    | name               | type       | entityType   | labelsOrTypes   | properties   |
|----+--------------------+------------+--------------+-----------------+--------------|
|  0 | unique_person_name | UNIQUENESS | NODE         | ['Person']      | ['name']     |
|  1 | unique_place_city  | UNIQUENESS | NODE         | ['Place']       | ['city']     |
+----+--------------------+------------+--------------+-----------------+--------------+
